# BERT
---
## Giriş

BERT ilk olarak, Jacob Devlin, Ming-Wei Chang, Kenton Lee ve Kristina Toutanova tarafından [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) adlı makalede sunulmuştur. Bidirectional bir transformer modelidir, Toronto Book Corpus ve Wikipedia'dan oluşan büyük bir corpus ile eğitilmiştir. Eğitimi için maskelenmiş dil modeli objective ve next sentence prediction kombinasyonu kullanılmıştır. 

Repo: [google-research/bert](https://github.com/google-research/bert) 

## Paper: Abstract
We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.

BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement).

## İpuçları

- BERT bünyesinde mutlak __pozisyon embedding__'leri barındırır. Bu yüzden "padding" işlemi cümlenin sağ tarafına doğru yapılmalıdır.
- BERT __masked language modeling__ (__MLM__) ve __next sentence prediction__ (__NSP__) görevleriyle eğitilmiştir. Maskelenmiş tokenleri tahmin etmekte ve __natural language understanding__'de (__NLU__) başarılıdır. Fakat metin üretmek için optimal değildir.

---

## BERT Text Classification (Non-English)

Dünya üzerinde 7.5 milyar insan ve 200'den fazla ulus yaşamaktadır ancak bunlardan sadece 1.2 milyarının ana dili İngilizce'dir. Bu nedenle, dünya üzerinde büyük miktarlarda İngilizce olmayan metin verisi olduğu söylenilebilir. 

Çoğu BERT tutorial'ı İngilizce dili üzerinden ilerler ve çeşitli problemleri İngilizce için nasıl çözüldüğünü anlatır. Bu yazıda farklı diller için eğitim adımlarından bahsedilecektir. 

Deep learning alanında şu sıralar populer bir tartışma var: Dil modeli çok dilli olmalı vs tek dilli olmalı. Aslında ikisi de yapılabilir. Çok dilli modeller, anlaşılacağı gibi, birden fazla dili anlama becerisine sahiptir. Örnek olarak Google Research tarafından sunulan mBERT verilebilir. Öte yandan tek dilli modeller, sadece bir dili anlayabilirler.

Çok dilli modeller, birçok modelde başarı gösterirler ancak boyut olarak büyüklerdir, eğitimleri için daha çok zaman ve veri gereklidir. Bu yüksek eğitim maliyeti anlamına gelir. Bu yüzden, bu yazıda tek dilli, İngilizce olmayan, BERT tabanlı ve çok etiketli bir sınıflandırıcı eğitimi örneği ele alınacaktır. 

### Tutorial
Bu yazıda Simple Transformer kütüphanesi kullanılacaktır. Bu aslında bir NLP kütüphanesidir ve HuggingFace Transformer kütüphanesi üzerine kurulmuştur. Transformer modellerini birkaç satırda fine-tune etmemize imkan tanır. Kullanılacak veri seti, Germeval 2019'dir: Almanca tweet'lerden oluşur. Bu veri seti ile saldırgan dilde yazılmış tweet'ler ayırt edilmeye çalışılacaktır. Tweet'ler 4 kategoriye bölünmüştür: `PROFANITY`, `INSULT`, `ABUSE` ve `OTHERS`. Bu veri seti üzerinde erişilen en yüksek skor. `0.7361`'dir. Adımlar aşağıdaki gibi özetlenmiştir:

- _Simple Transformer_ kütüphanesi kurulumu
- Pre-trained monolingual model seçimi
- Veri seti yükleme
- Fine-tune gerçekleştirme
- Sonuçların değerlendirilmesi
- Eğitilen modelin kaydedilmesi
- Modelin gerçek örnek ile denenmesi

### _Simple Transformers_ kurulumu
Kütüphane, `pip` üzerinden yüklenebilir:

In [17]:
!pip install simpletransformers


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


---

### Pre-trained monolingual model seçimi
Kurulumu yapılan kütüphane, HuggingFace Transformer kütüphanesi üzerine kuruludur. Bu yüzden HuggingFace Hub'daki Transformer Library içinde tanımlanmış bütün pre-trained modeller için çalışacaktır. Topluluk tarafından yüklenen modellerin listesine [link](https://huggingface.co/models) üzerinden ulaşılabilir.

Örnekte `distilbert-base-german-cased` modeli kullanılacaktır. Bu model BERT'in daha küçük, daha hızlı ve daha maliyetsiz versiyonudur. BERT'e göre parametre sayısı %40 daha azdır ve %60 daha hızlı çalışır. 

---

### Veri seti yükleme
Veri seti iki adet metin belgesinde saklanmıştır ve https://fz.h-da.de/iggsa/data adresinden indirilebilir. İndirilen veriler `pandas` ile bir DataFrame'e dönüştürülebilir:


In [34]:
import pandas as pd

class_list = ["INSULT", "ABUSE", "PROFANITY", "OTHER"]
df1 = pd.read_csv("../data/raw/germeval2019GoldLabelsSubtask1_2.txt",
                  sep='\t',
                  lineterminator="\n",
                  encoding="utf-8",
                  names=["tweet", "task1", "task2"])
df2 = pd.read_csv("../data/raw/germeval2019.training_subtask1_2_korrigiert.txt",
                  sep='\t',
                  lineterminator="\n",
                  encoding="utf-8",
                  names=["tweet", "task1", "task2"])

df = pd.concat([df1, df2])
df['task2'] = df['task2'].str.replace('\r', '')
df['pred_class'] = df.apply(
    lambda x: class_list.index(x['task2']),
    axis=1)

df = df[['tweet', 'pred_class']]
print(df.shape)
df.head()

(15418, 2)


,tweet,pred_class
0,@JanZimmHHB @mopo Komisch das die RealitÃ¤tsve...,0
1,@faznet @Gruene_Europa @SPDEuropa @CDU CDU ste...,1
2,"@DLFNachrichten Die Gesichter, Namen, Religion...",3
3,@welt Wie verwirrt muss man sein um sich zu we...,1
4,@hacker_1991 @torben_braga Weil die AfD den Fe...,1


Bir test veri seti olmadığı için, veri setinin %10'u test için ayrılabilir:

In [37]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)

print('train shape: ', train_df.shape)
print('test shape: ', test_df.shape)

train shape:  (13876, 2)
test shape:  (1542, 2)


---
### Pre-trained modelin yüklenmesi
Bu adımda, pre-trained modelin çalışmaya dahil edilmesi gösterilecektir. `ClassificationModel` sınıfına ait bir instance oluşturarak gerçekleştirilebilir. Bu instance şu parametreleri alır:,
- __mimari__: bu örnek için `"bert"`
- __pre-trained model__: `"distilbert-base-german-cased"`
- __sınıf sayısı__: `4`
- __hiperparametreler__: `train_args`

Hiperparametreler oldukça farklı şekilde inşa edilebilir. Detaylar için _Simple Transformer_ dökümantasyonuna bakılabilir. 

In [41]:
from simpletransformers.classification import ClassificationModel

# Hiperparametre tanımları
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 4,
}

# ClassificationModel instance'ı oluşturmak
model = ClassificationModel(
    "bert",
    "distilbert-base-german-cased",
    num_labels=4,
    use_cuda=False,
    args=train_args,
)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.5.sa_layer_norm.bias', 'vocab_projector.bias', 'distilbert.transformer.layer.3.sa_layer_norm.bias', 'distilbert.transformer.layer.4.attention.v_lin.bias', 'distilbert.transformer.layer.4.sa_layer_norm.weight', 'distilbert.transformer.layer.5.attention.out_lin.weight', 'distilbert.transformer.layer.1.attention.out_lin.bias', 'distilbert.transformer.layer.5.ffn.lin1.bias', 'distilbert.transformer.layer.3.ffn.lin1.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.3.attention.k_lin.weight', 'distilbert.transformer.layer.4.attention.k_lin.bias', 'distilbert.transformer.layer.3.ffn.lin2.bias', 'distilbert.transformer.layer.4.ffn.lin2.bias', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.2.output_layer_norm.weight', 'distilbert.embeddings.position_embe

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


--- 
### Modelin fine-tune edilmesi
Modeli fine-tune edebilmek için `model.train_model()` metodunun çağrılması gereklidir.

In [ ]:
model.train_model(train_df)

/Users/alihantadal/Documents/GitHub/transformers/venv/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/13876 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1735 [00:00<?, ?it/s]